<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City - Part 1</font></h1>

## 1. Download required libraries

In [1]:
!conda install -c conda-forge beautifulsoup4 lxml html5lib requests geocoder geopy folium --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



## 2. Download html page and put in a soup object

In [4]:
from bs4 import BeautifulSoup
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url, 'lxml')

## 3. Scrape html page and save data into an array

In [5]:
table = soup.find('table',{'class':'wikitable sortable'})
listdict = []
for row in table.find_all('tr'):
    elements = row.find_all('td')
    if not elements:
        continue
    postalCode, borough, neighborhood = [element.text.strip() for element in elements]
    if borough != 'Not assigned':
        if neighborhood != 'Not assigned':
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':neighborhood})
        else:
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':borough})

## 4. Create pandas dataframe from the array

In [6]:
import pandas as pd
# instantiate the dataframe
df = pd.DataFrame(listdict)

In [10]:
group_boroughs = df.groupby('PostalCode')['Borough'].apply(lambda x: set(x).pop())
group_neighborhoods = df.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
toronto_df = pd.DataFrame(list(zip(group_boroughs.index, group_boroughs, group_neighborhoods)))
toronto_df.columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [12]:
toronto_df.shape

(103, 3)